In [16]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
import matplotlib.pyplot as plt

In [17]:
# Transformations that are used in the dataset
trans = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [18]:
# Loading the MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./', train= True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root='./', train =False, transform=trans)


In [19]:
# We are splitting the training set into a training set and a validation set
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [20]:
# Hyperparameters for the model
num_epochs = 10
batch_size = 100
learning_rate = 0.001

In [22]:
# Creating dataloaders for the respective datasets 
train_loader = DataLoader ( dataset=  train_dataset, batch_size = batch_size, shuffle= True)
valid_loader = DataLoader(dataset =val_dataset, batch_size = batch_size,   shuffle =False )
test_loader = DataLoader ( dataset = test_dataset, batch_size= batch_size, shuffle = False)


In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [24]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(7 * 7 * 64, 1000),
            nn.Linear(1000, 10)
        )

    def forward(self, x):
        out = self.features(x)
        out = torch.flatten(out, 1) 
        out = self.classifier(out)
        return out

model = CNN().to(device)


In [25]:
loss_funcn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
total_step = len(train_loader)
loss_list = []
total = 0  
correct = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Doing the forward pass
        outputs = model(images)
        loss = loss_funcn(outputs, labels)
        loss_list.append(loss.item())
        
        # back propagation

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Computing accuracy
        total += labels.size(0)
        predicted = torch.max(outputs.data, 1)[1]
        correct += (predicted == labels).sum().item()

    # Calculate accuracy for the whole dataset
    accuracy = correct / total * 100

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch + 1, num_epochs,
                                                                   loss.item(), accuracy))

print('Accuracy on the whole dataset: {:.2f}%'.format(accuracy))


Epoch [1/10], Loss: 0.2750, Accuracy: 87.96%
Epoch [2/10], Loss: 0.1826, Accuracy: 90.66%
Epoch [3/10], Loss: 0.1348, Accuracy: 91.91%
Epoch [4/10], Loss: 0.0987, Accuracy: 92.70%
Epoch [5/10], Loss: 0.2379, Accuracy: 93.27%
Epoch [6/10], Loss: 0.1642, Accuracy: 93.70%
Epoch [7/10], Loss: 0.1089, Accuracy: 94.03%
Epoch [8/10], Loss: 0.1010, Accuracy: 94.29%
Epoch [9/10], Loss: 0.0700, Accuracy: 94.50%
Epoch [10/10], Loss: 0.1084, Accuracy: 94.68%
Accuracy on the whole dataset: 94.68%


In [27]:
# Evaluating the model on the validation set

model.eval()
valid_labels= []

valid_predictions =[]

with torch.no_grad():
    for images, labels in valid_loader:
        
        images= images.to(device)
        labels =labels.to(device )

        outputs = model(images )
        predicted=  torch.max( outputs.data, 1 )[1] 

        valid_labels.extend(labels.cpu().numpy())
        valid_predictions.extend(predicted.cpu().numpy())

valid_accuracy =( sum([1 for i, j in  zip(valid_labels, valid_predictions) if i ==j]) / len(valid_labels) ) * 100

valid_f= f1_score( valid_labels, valid_predictions, average='weighted')

valid_confusion_matrix =confusion_matrix(valid_labels, valid_predictions )

print('Validation Accuracy: {:.2f}%'.format(valid_accuracy))
print('Validation F1 Score: {:.4f}'.format(valid_f))
print('Validation Confusion Matrix:')
print(valid_confusion_matrix)

Validation Accuracy: 97.81%
Validation F1 Score: 0.9781
Validation Confusion Matrix:
[[1195    0    2    1    0    1    2    1    0    1]
 [   0 1360    5    0    1    0    0    4    2    0]
 [   1    6 1109    5    0   15   28    6    5    3]
 [   2    1    2 1200    0   10    1    2    4    3]
 [   1    4    2    0 1137    0    0    3    3   13]
 [   1    3    4    6    0 1032    3   10    3    3]
 [   7    0    6    0    1    4 1202    0    1    0]
 [   1    5    0    0    1   10    0 1209    3   11]
 [   3    0    1    1    0    1    5    0 1107    9]
 [   3    2    0    1    5    1    0    6    2 1186]]


In [28]:
# Evaluating the model on the testing set


model.eval()
testing_labels= []
testing_predictions =[]

with torch.no_grad():
    for images, labels in test_loader:
        images= images.to(device)
        labels =labels.to(device )

        outputs = model(images )
        predicted=  torch.max( outputs.data, 1 )[1] 

        testing_labels.extend(labels.cpu().numpy())
        testing_predictions.extend(predicted.cpu().numpy())


testing_accuracy= (sum([1 for i, j in zip( testing_labels, testing_predictions) if i == j]) / len( testing_labels)) * 100

testing_f1 =f1_score(testing_labels, testing_predictions, average  ='weighted' )

testing_precision= precision_score (testing_labels, testing_predictions, average ='weighted' )

testing_recall =recall_score (testing_labels,testing_predictions, average  ='weighted' )

testing_confusion_matrix= confusion_matrix( testing_labels, testing_predictions)

print('Test Accuracy: {:.2f}%'.format( testing_accuracy ))
print('Test F1 Score: {:.4f}'.format(testing_f1 ) )
print('Test Precision: {:.4f}'.format( testing_precision) )
print('Test Recall: {:.4f}'.format (testing_recall))
print('Test Confusion Matrix:' )
print(testing_confusion_matrix )


Test Accuracy: 98.01%
Test F1 Score: 0.9801
Test Precision: 0.9802
Test Recall: 0.9801
Test Confusion Matrix:
[[ 975    1    0    0    0    1    2    1    0    0]
 [   0 1130    3    1    0    0    1    0    0    0]
 [   0    1  998    0    0    9   13    7    3    1]
 [   4    0    0  977    0   21    0    3    3    2]
 [   0    1    0    0  965    0    1    0    2   13]
 [   2    2    6    9    0  866    2    3    1    1]
 [  10    4    4    0    1    4  932    0    2    1]
 [   1    6    3    1    2    4    0 1008    2    1]
 [   4    1    1    1    1    1    0    0  960    5]
 [   4    3    1    0    4    2    0    4    1  990]]


In [29]:
_PATH = "C:\\Users\\SWETHA"

In [30]:
# Saving the model
torch.save(model.state_dict(), _PATH + '\\conv_net_model.ckpt')